# Optimize Interface Film Position

Find most optimal position of the film on the substrate interface.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material from the list of available materials. Must be an interface.
1. Set notebook parameters in cell 1.2. below (or use the default values).
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete.
1. Scroll down to view results.

## Notes

- The optimization is performed on a 2D grid of x,y translations.
- Interface material must have atoms labeled "0" for the substrate and "1" for the film.


## 1. Prepare the Environment
### 1.1. Install Packages


In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("")


### 1.2. Set optimization parameters


In [ ]:
MATERIAL_INDEX = 0  # Index of the material to optimize
# Grid parameters
GRID_SIZE = (20, 20)  # Resolution of the x-y grid
GRID_RANGE_X = (-0.5, 0.5)  # Range to search in x direction
GRID_RANGE_Y = (-0.5, 0.5)  # Range to search in y direction
USE_CARTESIAN = False  # Whether to use Cartesian coordinates

# Visualization parameters
SHOW_3D_LANDSCAPE = False  # Whether to show 3D energy landscape
STRUCTURE_REPETITIONS = [3, 3, 1]  # Repetitions for structure visualization


## 3. Load Material
### 3.1. Make sure that loaded material is an interface material (atoms must have labels "0" for the substrate and "1" for the film)

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
interface_material = materials[MATERIAL_INDEX]


### 3.2. Visualize the Material


In [ ]:
from utils.visualize import visualize_materials

visualize_materials([interface_material], repetitions=STRUCTURE_REPETITIONS)
visualize_materials([interface_material], repetitions=STRUCTURE_REPETITIONS, rotation='-90x')

### 3.3. Optimize Film Position

In [ ]:
from mat3ra.made.tools.build.interface import get_optimal_film_displacement
from mat3ra.made.tools.modify import interface_displace_part
from mat3ra.made.tools.calculate.calculators import InterfaceMaterialCalculator
from mat3ra.made.tools.optimize import evaluate_calculator_on_xy_grid

calculator = InterfaceMaterialCalculator()

# Calculate energy landscape
xy_matrix, energy_matrix = evaluate_calculator_on_xy_grid(
    material=interface_material,
    calculator_function=calculator.get_energy,
    modifier=interface_displace_part,
    grid_size_xy=GRID_SIZE,
    grid_range_x=GRID_RANGE_X,
    grid_range_y=GRID_RANGE_Y,
    use_cartesian_coordinates=USE_CARTESIAN
)

# Find optimal position
optimal_displacement = get_optimal_film_displacement(
    material=interface_material,
    calculator=calculator,
    grid_size_xy=GRID_SIZE,
    grid_range_x=GRID_RANGE_X,
    grid_range_y=GRID_RANGE_Y,
    use_cartesian_coordinates=USE_CARTESIAN
)

print(f"\nOptimal displacement vector: {optimal_displacement}")


## 4. Visualize Results
### 4.1. Plot Energy Landscape

In [ ]:
from mat3ra.utils.jupyterlite.plot import plot_2d_heatmap, plot_3d_surface

# Plot energy landscape
x_values, y_values = xy_matrix
plot_2d_heatmap(x_values, y_values, energy_matrix, optimal_displacement[:2])

if SHOW_3D_LANDSCAPE:
    plot_3d_surface(x_values, y_values, energy_matrix, optimal_displacement[:2])

# Create optimized material
optimized_material = interface_displace_part(
    interface_material,
    displacement=optimal_displacement,
    use_cartesian_coordinates=USE_CARTESIAN
)


### 4.1. Visualize Original and Optimized Materials

In [ ]:
print("\nVisualization of original and optimized materials:")
visualize_materials([interface_material, optimized_material],
                    repetitions=STRUCTURE_REPETITIONS)
visualize_materials([interface_material, optimized_material],
                    repetitions=STRUCTURE_REPETITIONS,
                    rotation='-90x')


## 5. Save Results


In [ ]:
from utils.jupyterlite import set_materials

optimized_material.name = f"{interface_material.name} Optimized XY"
set_materials(optimized_material)